### 2 modules: array, and nn

In [1]:
import ml.array as ml
import ml.nn as nn

### Numpy-like syntax

In [2]:
a = ml.Array([1,2,3,4])
b = ml.Array([5,6,7,8])

out = (a + b).sqrt().sum()
print(out)

array(11.904297, dtype=float32)


### View computation graphs

In [3]:
out.view_graph()

### Differentiation

In [2]:
a = ml.rand([2,4])
b = ml.rand([4,2])
c = ml.rand([2])
out = (a @ b).sqrt().sum()

# backward 
out.build_backward()
print(a.grad().eval())
print(b.grad().eval())


[[0.26133835 0.6266034  0.435673   0.48981383]
 [0.25419953 0.6002871  0.42334855 0.47057244]]
[[0.8678334  0.59258586]
 [0.64149284 0.44166332]
 [0.39418766 0.27127358]
 [0.9229811  0.63164824]]


In [5]:
out.view_graph()

### Machine learning: Iris dataset

In [160]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def get_data():
    iris = load_iris()
    X, y = iris.data, iris.target
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train = ml.Array(X_train.tolist())
    X_test = ml.Array(X_test.tolist())
    y_train = ml.Array(y_train.tolist())
    y_test = ml.Array(y_test.tolist())
    return X_train, X_test, y_train, y_test

def test_accuracy(W,b):
    X_train, X_test, y_train, y_test = get_data()
    logits_test = X_test.eval() @ W.eval() + b.eval()
    predictions = np.argmax(logits_test, axis=1)
    accuracy = np.mean((predictions == y_test.eval()))
    print(f'Accuracy: {accuracy.item()}')

In [167]:
class Linear:

    def __init__(self, inp_shape, out_shape):
        self.w = ml.rand([inp_shape,out_shape])
        self.b = ml.rand([out_shape])
        self.params = [self.w, self.b]

    def __call__(self, x):
        return x @ self.w + self.b


def train(lr, iter):
    X_train, X_test, y_train, y_test = get_data()
    one_hot_train = nn.one_hot_encode(y_train, 3)
    
    l = Linear(4,3)
    logits = l(X_train)

    # built in loss functions
    loss = nn.softmax_cross_entropy_loss(logits, one_hot_train) 
    loss.build_backward()

    # optimizers supported: Adam, SGD
    optim = nn.Adam(lr, l.params)

    # training loop
    for it in range(iter):
        if (it % (iter//10) == 0): print(f"{it} loss = ", loss.eval().item())
        loss.eval()
        loss.zero_grad()
        loss.set_reeval()
        for p in l.params:
            p.grad().set_reeval()
        optim.step()
    
    test_accuracy(*l.params)
    return loss

lr = 0.01
iter = 100

loss = train(lr,iter)

0 loss =  1.0898603200912476
10 loss =  0.3756318688392639
20 loss =  0.24680905044078827
30 loss =  0.176712304353714
40 loss =  0.14138753712177277
50 loss =  0.12131211161613464
60 loss =  0.10833719372749329
70 loss =  0.09953636676073074
80 loss =  0.09285078197717667
90 loss =  0.08762286603450775
Accuracy: 0.9666666666666667


In [8]:
# view neural network graph
loss.view_graph(view_data=False)